# Clustering Crypto

In [255]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [256]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
df = pd.read_csv(file_path)

df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [257]:
df.count()

Unnamed: 0         1252
CoinName           1252
Algorithm          1252
IsTrading          1252
ProofType          1252
TotalCoinsMined     744
TotalCoinSupply    1252
dtype: int64

In [258]:
# Keep all the cryptocurrencies that are being traded.
df = df[(df["IsTrading"] == True)]

In [259]:
# Keep all the cryptocurrencies that have a working algorithm.
df.set_index('Unnamed: 0', inplace=True)
df.rename_axis(index=None, inplace=True)
df.dropna(subset = ['Algorithm'])
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [260]:
df.count()

CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [261]:
# Remove the "IsTrading" column. 
df.drop(columns=["IsTrading"], inplace=True)
df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [262]:
# Remove rows that have at least 1 null value.
df = df.dropna()
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [263]:
df.dtypes

CoinName            object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [264]:
# Keep the rows where coins are mined.
#df = df[df.TotalCoinSupply != "0"]
df = df[df.TotalCoinsMined > 0]
df.count()

CoinName           532
Algorithm          532
ProofType          532
TotalCoinsMined    532
TotalCoinSupply    532
dtype: int64

In [265]:
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [266]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame(data = df["CoinName"])
names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [267]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns=["CoinName"], inplace=True)
df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [268]:
# Use get_dummies() to create variables for text features.
crypto_encoded = pd.get_dummies(df, columns=["Algorithm", "ProofType"])
crypto_encoded.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [269]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(crypto_encoded)
print(X[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [270]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA model
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(X)

In [271]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=crypto_pca, index=names_df.index, columns=["PC 1", "PC 2", "PC 3"])

pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.340558,0.885901,-0.484038
404,-0.323901,0.886268,-0.484268
1337,2.309960,1.549106,-0.637188
BTC,-0.141433,-1.250096,0.176665
ETH,-0.151990,-1.938223,0.338250
LTC,-0.171738,-1.136405,0.001670
DASH,-0.385807,1.106019,-0.411192
XMR,-0.135258,-2.191413,0.379828
ETC,-0.150433,-1.938298,0.338232
ZEC,-0.165952,-2.063660,0.345377


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [272]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1, 11))


# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)    

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [273]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[1 1 1 2 2 2 1 2 2 2 1 2 1 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 1 2 1 2 2 1 1 2
 2 2 2 2 2 1 1 2 2 2 2 2 1 1 2 1 2 2 2 2 1 2 2 1 2 1 1 1 2 2 2 1 1 1 1 1 2
 2 2 1 1 2 1 2 1 1 2 2 2 2 1 1 2 1 2 2 1 1 2 1 1 2 2 1 1 2 1 1 2 1 2 1 2 1
 2 1 1 2 2 1 2 2 2 1 2 2 2 2 2 1 1 2 2 2 1 2 1 2 2 1 2 1 2 1 1 2 2 1 2 2 1
 1 2 1 2 1 1 1 2 2 2 2 1 1 1 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 2 1 2 1 1 2 1 2
 1 1 2 1 2 1 2 1 2 1 1 1 1 2 1 1 1 1 1 2 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1
 1 1 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 1 2 1 1 2 1 2 2 1 2 2 1 2 1 1 1 2 1 1 2
 1 1 1 1 1 1 1 2 1 2 1 1 1 1 2 1 2 1 2 2 2 2 1 2 1 1 2 1 2 2 2 1 2 1 2 2 2
 1 2 1 2 1 1 1 2 1 2 2 2 2 2 1 1 2 1 1 1 2 1 2 1 2 1 2 1 1 1 1 2 1 1 2 1 1
 1 2 2 2 2 1 1 1 1 2 1 2 2 2 1 1 2 2 1 1 2 1 2 2 2 1 2 2 1 1 1 2 2 2 1 1 1
 2 2 1 2 2 2 2 1 0 0 2 2 2 1 0 1 1 1 1 2 2 2 2 1 1 1 2 1 2 1 1 1 1 2 1 1 2
 1 1 2 2 1 2 1 2 2 2 2 1 1 2 1 2 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 2 1 2 2 2 2
 1 1 1 1 2 1 1 2 1 1 2 0 2 1 2 2 2 1 2 1 2 2 1 2 2 1 2 1 2 1 1 2 1 1 1 1 1
 2 2 2 1 1 1 2 1 2 1 2 1 

In [274]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
pcs_df["Class"] = model.labels_

clustered_df = pd.concat([names_df, df, pcs_df], axis=1)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.340558,0.885901,-0.484038,1
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.323901,0.886268,-0.484268,1
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.309960,1.549106,-0.637188,1
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.141433,-1.250096,0.176665,2
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.151990,-1.938223,0.338250,2
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.171738,-1.136405,0.001670,2
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.385807,1.106019,-0.411192,1
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.135258,-2.191413,0.379828,2
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.150433,-1.938298,0.338232,2
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000,-0.165952,-2.063660,0.345377,2


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [275]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class")
fig.update_layout(legend=dict(x=0,y=1))

In [281]:
# Create a table with tradable cryptocurrencies.
hvplot_table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)
hvplot_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [276]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df.CoinName)} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [277]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

encoded_df = pd.DataFrame(
    data=clustered_df, index=clustered_df.index, columns=['TotalCoinSupply', 'TotalCoinsMined'])

#encoded_df = clustered_df.drop(columns=['CoinName', 'Algorithm', 'ProofType', 'Class'], inplace=True)

crypto_scaled = MinMaxScaler().fit_transform(encoded_df)
crypto_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [278]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data=crypto_scaled, index=clustered_df.index, columns=['TotalCoinSupply', 'TotalCoinsMined'])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [280]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)